# Make a MODFLOW 6 model with FloPy 4

In this notebook we will make MODFLOW6 models using FloPy 3 and FloPy4. The process for making models with FloPy 4 is roughly the same, except that packages are made before models and models are made before the simulation. Remember in FloPy 3 that the simulation is made first, followed by a model(s), and then its packages.

FloPy 4 has functionality for reading binary dependent variable output as xarray arrays. We will use this functionality to plot simulated model results for models generated with FloPy 3 and FloPy 4.

In this example, list-based stress packages will be generated.

In [ ]:
%matplotlib inline
import flopy
import flopy4
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

from plot_results import plot_heads

In [ ]:
# Create workspace
workspace = (Path("./twri") / "list_stresspkg").resolve()
workspace.mkdir(parents=True, exist_ok=True)

name = "twri"


## Temporal discretization

In [ ]:
# Timing
time = flopy4.mf6.utils.time.Time.from_timestamps(
    ["2000-01-01", "2000-01-02", "2000-01-03", "2000-01-04"]
)
nper = time.nper

tdis = flopy4.mf6.simulation.Tdis.from_time(time)

perioddata = [
    (float(perlen), int(nstp), float(tsmult)) 
    for perlen, nstp, tsmult in 
    zip(tdis.perlen.values, tdis.nstp.values, tdis.tsmult.values)
    ]


## Spatial discretization

In [ ]:
# Grid
nlay = 3
nrow = 15
ncol = 15
shape = (nlay, nrow, ncol)
nodes = np.prod(shape)
delr = np.ones((ncol,)) * 5000.0
delc = np.ones((nrow,)) * 5000.0
idomain = np.ones(shape, dtype=int)
top = np.full((nrow, ncol), 200.0, dtype=float)
bottom = np.stack([np.full((nrow, ncol), val) for val in [-200.0, -300.0, -450.0]])
grid = flopy4.mf6.utils.grid.StructuredGrid(
    nlay=nlay, nrow=nrow, ncol=ncol, top=top, botm=bottom, delr=delr, delc=delc, idomain=idomain
)
dims = {"nper": nper, "ncpl": nrow * ncol, **dict(grid.dataset.sizes)}  # TODO: temporary



## FloPy 3 simulation, time discretization, and gwf model

In [ ]:
workspace3 = workspace / "flopy3"
sim3 = flopy.mf6.MFSimulation(sim_name=name, sim_ws=workspace3)
tdis3 = flopy.mf6.ModflowTdis(sim3, nper=nper, perioddata=perioddata)
gwf3 = flopy.mf6.ModflowGwf(sim3, modelname=name)

## Discretization Package

In [ ]:
# Grid discretization
# TODO: xorigin, yorigin
dis = flopy4.mf6.gwf.Dis.from_grid(grid=grid)

dis3 = flopy.mf6.ModflowGwfdis(
    gwf3, 
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    top=top,
    botm=bottom,
    idomain=idomain,
    )

## Internal Packages

In [ ]:
# Node properties
icelltype = np.stack([np.full((nrow, ncol), val) for val in [1, 0, 0]])
k = np.stack([np.full((nrow, ncol), val) for val in [1.0e-3, 1.0e-4, 2.0e-4]])
k33 = np.stack([np.full((nrow, ncol), val) for val in [2.0e-8, 2.0e-8, 2.0e-8]])

npf = flopy4.mf6.gwf.Npf(
    # TODO: no need for reshaping once array structuring converter is done
    icelltype=icelltype.reshape((nodes,)),
    k=k.reshape((nodes,)),
    k33=k33.reshape((nodes,)),
    cvoptions=flopy4.mf6.gwf.Npf.CvOptions(dewatered=True),
    perched=True,
    save_flows=True,
    dims=dims,
)

npf3 = flopy.mf6.ModflowGwfnpf(
    gwf3,
    cvoptions="dewatered",
    perched=True,
    icelltype=icelltype,
    k=k,
    k33=k33,
)

In [ ]:
# Storage
sto = flopy4.mf6.gwf.Sto(
    storagecoefficient=False,
    ss=1.0e-5,
    sy=0.15,
    steady_state=[True, False, False],
    iconvert=0,
    dims=dims,
)

sto3 = flopy.mf6.ModflowGwfsto(
    gwf3,
    steady_state={0: True},
    transient={1: True},
    iconvert=0,
    ss=1e-5,
    sy=0.15,
)

In [ ]:
# Initial conditions
ic = flopy4.mf6.gwf.Ic(strt=0.0, dims=dims)

ic3 = flopy.mf6.ModflowGwfic(
    gwf3,
    strt=0.0,
)


## Stress packages

In [ ]:
# Uniform recharge on the top layer
rch_rate = np.full((nlay, nrow, ncol), flopy4.mf6.constants.FILL_DNODATA)
rate = np.repeat(np.expand_dims(rch_rate, axis=0), repeats=nper, axis=0)
rate[0, 0, ...] = 3.0e-8

rch = flopy4.mf6.gwf.Rch(recharge=rate.reshape(nper, -1), dims=dims)

rch3 = flopy.mf6.ModflowGwfrcha(
    gwf3,
    recharge=3e-8,
)

In [ ]:
# Constant head boundary on the left
chd = flopy4.mf6.gwf.Chd(
    head={"*": {(k, i, 0): 0.0 for k in range(nlay - 1) for i in range(nrow)}},
    print_input=True,
    print_flows=True,
    save_flows=True,
    dims=dims,
)


chd_spd = [((k, i, 0), 0.0) for k in range(nlay - 1) for i in range(nrow)]
chd3 = flopy.mf6.ModflowGwfchd(
    gwf3,
    maxbound=len(chd_spd),
    stress_period_data=chd_spd,
    print_input=True,
    print_flows=True,
    save_flows=True,
)


In [ ]:
# Drain in the center left of the model
elevation = [0.0, 0.0, 10.0, 20.0, 30.0, 50.0, 70.0, 90.0, 100.0]
conductance = 1.0
drn = flopy4.mf6.gwf.Drn(
    elev={"*": {(0, 7, j + 1): elevation[j] for j in range(9)}},
    cond={"*": {(0, 7, j + 1): conductance for j in range(9)}},
    print_input=True,
    print_flows=True,
    save_flows=True,
    dims=dims,
)

drn_spd = [((0, 7, j + 1), elevation[j], conductance) for j in range(9)]
drn3 = flopy.mf6.ModflowGwfdrn(
    gwf3,
    maxbound=len(drn_spd),
    stress_period_data={0: drn_spd},
)


In [ ]:
# Wells scattered throughout the model
wel_q = -5.0
wel_nodes = [
    [2, 4, 10],
    [1, 3, 5],
    [1, 5, 11],
    [0, 8, 7],
    [0, 8, 9],
    [0, 8, 11],
    [0, 8, 13],
    [0, 10, 7],
    [0, 10, 9],
    [0, 10, 11],
    [0, 10, 13],
    [0, 12, 7],
    [0, 12, 9],
    [0, 12, 11],
    [0, 12, 13],
]
wel = flopy4.mf6.gwf.Wel(
    q={"*": {(layer, row, col): wel_q for layer, row, col in wel_nodes}},
    dims=dims,
)

wel_spd = [((k, i, j), wel_q) for k, i, j in wel_nodes]
wel3 = flopy.mf6.ModflowGwfwel(
    gwf3,
    maxbound=len(wel_spd),
    stress_period_data=wel_spd,
)


## Output control

In [ ]:
# Output control
oc = flopy4.mf6.gwf.Oc(
    budget_file="gwf.bud",
    head_file="gwf.hds",
    save_head={0: "all"},
    save_budget={0: "all"},
    dims=dims,
)

oc3 = flopy.mf6.ModflowGwfoc(
    gwf3,
    budget_filerecord=f"{name}.bud",
    head_filerecord=f"{name}.hds",
    saverecord=[("head", "all"), ("budget", "all")]
)


## FloPy 4 groundwater model

In [ ]:
# Flow model
gwf = flopy4.mf6.gwf.Gwf(
    dis=grid,
    ic=ic,
    npf=npf,
    sto=sto,
    oc=oc,
    chd=[chd],
    rch=[rch],
    wel=[wel],
    drn=[drn],
    dims=dims,
)



## IMS solver package

In [ ]:
# Solver
print_option = "summary"
outer_dvclose = 1.0e-4
outer_maximum = 500
inner_dvclose = 1.0e-4
inner_rclose = 0.001
inner_maximum = 100
linear_acceleration = "cg"
relaxation_factor = 0.97


ims = flopy4.mf6.Ims(
    print_option=print_option,
    outer_dvclose=outer_dvclose,
    outer_maximum=outer_maximum,
    inner_dvclose=inner_dvclose,
    inner_rclose=inner_rclose,
    inner_maximum=inner_maximum,
    linear_acceleration=linear_acceleration,
    relaxation_factor=relaxation_factor,
    models=["gwf"],
)

ims3 = flopy.mf6.ModflowIms(
    sim3,
    print_option=print_option,
    outer_dvclose=outer_dvclose,
    outer_maximum=outer_maximum,
    inner_dvclose=inner_dvclose,
    rcloserecord=inner_rclose,
    inner_maximum=inner_maximum,
    linear_acceleration=linear_acceleration,
    relaxation_factor=relaxation_factor,
)

sim3.register_ims_package(ims3, [name])


## FloPy 4 simulation package

In [ ]:
# Create simulation
workspace4 = workspace / "flopy4"
workspace4.mkdir(parents=True, exist_ok=True)

sim = flopy4.mf6.simulation.Simulation(
    name="twri",
    tdis=tdis,
    models={"gwf": gwf},
    solutions={"ims": ims},
    workspace=workspace4,
)

## Write input files using FloPy 4 and run model

In [ ]:
# Write input files and run the simulation
sim.write()
sim.run()  # assumes the ``mf6`` executable is available on your PATH.

## Write input files using FloPy 3 and run model

In [ ]:
sim3.write_simulation()
sim3.run_simulation()

## Load and plot simulated results

In [ ]:
# Load head results
head4 = flopy4.mf6.utils.open_hds(
    workspace4 / f"{gwf.name}.hds",
    workspace4 / f"{gwf.name}.dis.grb",
)



In [ ]:
# Load head results
head3 = flopy4.mf6.utils.open_hds(
    workspace3 / f"{name}.hds",
    workspace3 / f"{name}.dis.grb",
)



In [ ]:
# Plot head results
plot_heads(head4, head3, workspace)